In [1]:
import pandas as pd

In [2]:
TARGET_COLUMNS = [ 
    'hospital_ccn',
    'hospital_ein',
    'code_type',
    'internal_code',
    'description',
    'code_orig',
    'code_prefix',
    'code',
    'modifier',
    'ndc',
    'ndc_num_hcpcs_doses',
    'ndc_quantity_desc',
    'apc',
    'rev_code',
    'rev_desc',
    'billing_class',
    'patient_class',
    'payer_category',
    'payer_orig',
    'payer_name',
    'plan_orig',
    'plan_name',
    'plan_id',
    'plan_type',
    'rate',
    'rate_method',
    'rate_desc',
    'filename',
    'file_last_updated',
    'url',
    'permalink'
]

In [3]:
ccn = "030037"
url = "https://content.steward.org/sites/default/files/2023-01/621795587_TempeStLukesHospital_standardcharges.csv"
filename = url.split("/")[-1]
ein = filename.split("_")[0]
ein = ein[:2] + "-" + ein[2:]

In [4]:
import subprocess
subprocess.run(["wget", "--no-clobber", url, "-O", filename])

File ‘621795587_TempeStLukesHospital_standardcharges.csv’ already there; not retrieving.


CompletedProcess(args=['wget', '--no-clobber', 'https://content.steward.org/sites/default/files/2023-01/621795587_TempeStLukesHospital_standardcharges.csv', '-O', '621795587_TempeStLukesHospital_standardcharges.csv'], returncode=1)

In [5]:
df_in = pd.read_csv(filename, dtype={'Rev Code': str, 'DRG': str}, low_memory=False)
df_in

,As of Date,Line Type,Line ID,Charge Code/Package,Charge Description,DRG,Patient Type,CPT,Modifiers,Rev Code,...,Tricare HNFS,TriWest VA PCCC,United Behavioral Health,United Healthcare Compass HIX,United Healthcare Evercare AHCCCS,United Healthcare Medicare Advantage,United Healthcare No Options (All Other),University Physicians Healthcare Medicare Advantage,Workers Comp 100,Workers Comp 50
0,1/13/2023,CDM,1,1000114S,CO-PILOT VALVE,NaN,OP,NaN,,272,...,150.00,NaN,150.00,71.55,NaN,NaN,71.55,NaN,150.00,75.00
1,1/13/2023,CDM,2,1000284S,SET LAP/CHOLANGIO,NaN,OP,NaN,,272,...,681.30,NaN,681.30,324.98,NaN,NaN,324.98,NaN,681.30,340.65
2,1/13/2023,CDM,3,1000564S,DRAINAGE BAG W/TUBING 600mL,NaN,OP,NaN,,272,...,97.50,NaN,97.50,46.51,NaN,NaN,46.51,NaN,97.50,48.75
3,1/13/2023,CDM,4,1000823S,INSTAT MCH MICROFIBLR CLGN 1GM,NaN,OP,NaN,,272,...,776.50,NaN,776.50,370.39,NaN,NaN,370.39,NaN,776.50,388.25
4,1/13/2023,CDM,5,1001512S,STAYFIX SMALL,NaN,OP,NaN,,272,...,65.63,NaN,65.63,31.31,NaN,NaN,31.31,NaN,65.63,32.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19301,1/13/2023,DRG,19302,APR-DRG 952-2,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-2,IP,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7594.67,NaN,NaN,NaN,NaN,NaN
19302,1/13/2023,DRG,19303,APR-DRG 952-3,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-3,IP,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,12437.74,NaN,NaN,NaN,NaN,NaN
19303,1/13/2023,DRG,19304,APR-DRG 952-4,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-4,IP,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,24634.93,NaN,NaN,NaN,NaN,NaN
19304,1/13/2023,DRG,19305,APR-DRG 955-1,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,955-1,IP,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_mid = pd.DataFrame(df_in)
df_mid = df_mid.rename(columns={
    'Line Type': 'code_type',
    'As of Date': 'file_last_updated',
    'Line ID': 'internal_code',
    'Charge Code/Package': 'code_orig',
    'Charge Description': 'description',
    'DRG': 'drg',
    'CPT': 'cpt',
    'Patient Type': 'patient_class',
    'Modifiers': 'modifier',
    'Rev Code': 'rev_code',
})

df_mid

,file_last_updated,code_type,internal_code,code_orig,description,drg,patient_class,cpt,modifier,rev_code,...,Tricare HNFS,TriWest VA PCCC,United Behavioral Health,United Healthcare Compass HIX,United Healthcare Evercare AHCCCS,United Healthcare Medicare Advantage,United Healthcare No Options (All Other),University Physicians Healthcare Medicare Advantage,Workers Comp 100,Workers Comp 50
0,1/13/2023,CDM,1,1000114S,CO-PILOT VALVE,NaN,OP,NaN,,272,...,150.00,NaN,150.00,71.55,NaN,NaN,71.55,NaN,150.00,75.00
1,1/13/2023,CDM,2,1000284S,SET LAP/CHOLANGIO,NaN,OP,NaN,,272,...,681.30,NaN,681.30,324.98,NaN,NaN,324.98,NaN,681.30,340.65
2,1/13/2023,CDM,3,1000564S,DRAINAGE BAG W/TUBING 600mL,NaN,OP,NaN,,272,...,97.50,NaN,97.50,46.51,NaN,NaN,46.51,NaN,97.50,48.75
3,1/13/2023,CDM,4,1000823S,INSTAT MCH MICROFIBLR CLGN 1GM,NaN,OP,NaN,,272,...,776.50,NaN,776.50,370.39,NaN,NaN,370.39,NaN,776.50,388.25
4,1/13/2023,CDM,5,1001512S,STAYFIX SMALL,NaN,OP,NaN,,272,...,65.63,NaN,65.63,31.31,NaN,NaN,31.31,NaN,65.63,32.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19301,1/13/2023,DRG,19302,APR-DRG 952-2,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-2,IP,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7594.67,NaN,NaN,NaN,NaN,NaN
19302,1/13/2023,DRG,19303,APR-DRG 952-3,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-3,IP,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,12437.74,NaN,NaN,NaN,NaN,NaN
19303,1/13/2023,DRG,19304,APR-DRG 952-4,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-4,IP,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,24634.93,NaN,NaN,NaN,NaN,NaN
19304,1/13/2023,DRG,19305,APR-DRG 955-1,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,955-1,IP,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_mid['code_type'] = 'none'
df_mid.loc[df_mid['cpt'].notnull(), 'code_type'] = 'cpt'
df_mid.loc[df_mid['drg'].notnull(), 'code_type'] = 'drg'
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,drg,patient_class,cpt,modifier,rev_code,...,Tricare HNFS,TriWest VA PCCC,United Behavioral Health,United Healthcare Compass HIX,United Healthcare Evercare AHCCCS,United Healthcare Medicare Advantage,United Healthcare No Options (All Other),University Physicians Healthcare Medicare Advantage,Workers Comp 100,Workers Comp 50
0,1/13/2023,none,1,1000114S,CO-PILOT VALVE,NaN,OP,NaN,,272,...,150.00,NaN,150.00,71.55,NaN,NaN,71.55,NaN,150.00,75.00
1,1/13/2023,none,2,1000284S,SET LAP/CHOLANGIO,NaN,OP,NaN,,272,...,681.30,NaN,681.30,324.98,NaN,NaN,324.98,NaN,681.30,340.65
2,1/13/2023,none,3,1000564S,DRAINAGE BAG W/TUBING 600mL,NaN,OP,NaN,,272,...,97.50,NaN,97.50,46.51,NaN,NaN,46.51,NaN,97.50,48.75
3,1/13/2023,none,4,1000823S,INSTAT MCH MICROFIBLR CLGN 1GM,NaN,OP,NaN,,272,...,776.50,NaN,776.50,370.39,NaN,NaN,370.39,NaN,776.50,388.25
4,1/13/2023,none,5,1001512S,STAYFIX SMALL,NaN,OP,NaN,,272,...,65.63,NaN,65.63,31.31,NaN,NaN,31.31,NaN,65.63,32.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19301,1/13/2023,drg,19302,APR-DRG 952-2,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-2,IP,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7594.67,NaN,NaN,NaN,NaN,NaN
19302,1/13/2023,drg,19303,APR-DRG 952-3,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-3,IP,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,12437.74,NaN,NaN,NaN,NaN,NaN
19303,1/13/2023,drg,19304,APR-DRG 952-4,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-4,IP,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,24634.93,NaN,NaN,NaN,NaN,NaN
19304,1/13/2023,drg,19305,APR-DRG 955-1,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,955-1,IP,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_mid['code_type'].value_counts()

cpt     9529
none    7539
drg     2238
Name: code_type, dtype: int64

In [9]:
money_columns = df_mid.columns.to_list()[10:]
remaining_columns = df_mid.columns.to_list()[:10]
df_mid = pd.melt(df_mid, id_vars=remaining_columns, var_name='payer_orig', value_name='rate')
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,drg,patient_class,cpt,modifier,rev_code,payer_orig,rate
0,1/13/2023,none,1,1000114S,CO-PILOT VALVE,NaN,OP,NaN,,272,Gross Charges,150.0
1,1/13/2023,none,2,1000284S,SET LAP/CHOLANGIO,NaN,OP,NaN,,272,Gross Charges,681.3
2,1/13/2023,none,3,1000564S,DRAINAGE BAG W/TUBING 600mL,NaN,OP,NaN,,272,Gross Charges,97.5
3,1/13/2023,none,4,1000823S,INSTAT MCH MICROFIBLR CLGN 1GM,NaN,OP,NaN,,272,Gross Charges,776.5
4,1/13/2023,none,5,1001512S,STAYFIX SMALL,NaN,OP,NaN,,272,Gross Charges,65.63
...,...,...,...,...,...,...,...,...,...,...,...,...
1312803,1/13/2023,drg,19302,APR-DRG 952-2,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-2,IP,NaN,NaN,NaN,Workers Comp 50,NaN
1312804,1/13/2023,drg,19303,APR-DRG 952-3,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-3,IP,NaN,NaN,NaN,Workers Comp 50,NaN
1312805,1/13/2023,drg,19304,APR-DRG 952-4,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-4,IP,NaN,NaN,NaN,Workers Comp 50,NaN
1312806,1/13/2023,drg,19305,APR-DRG 955-1,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,955-1,IP,NaN,NaN,NaN,Workers Comp 50,NaN


In [10]:
money_columns

['Gross Charges',
 'Discounted Cash Price',
 'De-Identified Minimum',
 'De-Identified Maximum',
 'APIPA Medicaid AHCCCS',
 'Arizona Foundation',
 'Arizona Medical Network',
 'Arizona Medical Network EPO',
 'Arizona Medical Network PPO',
 'AZ Complete Care Exchange (FKA) Health Net Exchange HIX',
 'AZ Complete Care Medicare FKA Health Net Medicare',
 'AZ Priority Care',
 'Banner University Family Care AHCCCS',
 'BCBS of AZ Commercial',
 'Bridgeway Medicare Advantage',
 'Care1st Health Plan AHCCCS',
 'CareMore Medicare Advantage',
 'CARESACT',
 'Cigna Behavioral Health AKA Evernorth Behavioral Health',
 'Cigna Commercial',
 'Cigna Medicare',
 'City of Tempe',
 'Commercial Non-Contracted',
 'Coventry Health Care International Payor Access',
 'Devoted Health',
 'Flat Rate ER',
 'Gallager Bassett',
 'GEO Federal Inmates',
 'Global Excel',
 'Health Choice Arizona AHCCCS',
 'Health Choice AZ Medicare Advantage',
 'Health Choice Integrated Care (HCIC)',
 'Health Net Commercial',
 'Humana Comme

In [11]:
df_mid['rev_code'] = df_mid['rev_code'].fillna('na')

def pad_rev_code_if_needed(rev_code):
    if type(rev_code) == str and rev_code != 'na':
        if len(rev_code) == 3:
            return '0' + rev_code
        elif len(rev_code) == 2:
            return '00' + rev_code
        elif len(rev_code) == 1:
            return '000' + rev_code

    return rev_code

df_mid['rev_code'] = df_mid['rev_code'].apply(pad_rev_code_if_needed)
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,drg,patient_class,cpt,modifier,rev_code,payer_orig,rate
0,1/13/2023,none,1,1000114S,CO-PILOT VALVE,NaN,OP,NaN,,0272,Gross Charges,150.0
1,1/13/2023,none,2,1000284S,SET LAP/CHOLANGIO,NaN,OP,NaN,,0272,Gross Charges,681.3
2,1/13/2023,none,3,1000564S,DRAINAGE BAG W/TUBING 600mL,NaN,OP,NaN,,0272,Gross Charges,97.5
3,1/13/2023,none,4,1000823S,INSTAT MCH MICROFIBLR CLGN 1GM,NaN,OP,NaN,,0272,Gross Charges,776.5
4,1/13/2023,none,5,1001512S,STAYFIX SMALL,NaN,OP,NaN,,0272,Gross Charges,65.63
...,...,...,...,...,...,...,...,...,...,...,...,...
1312803,1/13/2023,drg,19302,APR-DRG 952-2,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-2,IP,NaN,NaN,na,Workers Comp 50,NaN
1312804,1/13/2023,drg,19303,APR-DRG 952-3,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-3,IP,NaN,NaN,na,Workers Comp 50,NaN
1312805,1/13/2023,drg,19304,APR-DRG 952-4,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-4,IP,NaN,NaN,na,Workers Comp 50,NaN
1312806,1/13/2023,drg,19305,APR-DRG 955-1,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,955-1,IP,NaN,NaN,na,Workers Comp 50,NaN


In [12]:
df_mid_cpt = pd.DataFrame(df_mid.loc[df_mid['code_type'] == 'cpt'])
df_mid_cpt['code'] = df_mid['cpt']
df_mid_cpt['code_prefix'] = 'hcpcs_cpt'
df_mid_cpt = df_mid_cpt[~df_mid_cpt['code'].str.contains(".", regex=False)]

In [13]:
df_mid_drg = pd.DataFrame(df_mid.loc[df_mid['code_type'] == 'drg'])
df_mid_drg['code'] = df_mid_drg['drg']
df_mid_drg

,file_last_updated,code_type,internal_code,code_orig,description,drg,patient_class,cpt,modifier,rev_code,payer_orig,rate,code
17068,1/13/2023,drg,17069,MS-DRG 001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,1,IP,NaN,NaN,na,Gross Charges,846592.03,1
17069,1/13/2023,drg,17070,MS-DRG 002,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,2,IP,NaN,NaN,na,Gross Charges,752886.92,2
17070,1/13/2023,drg,17071,MS-DRG 003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,3,IP,NaN,NaN,na,Gross Charges,645099.41,3
17071,1/13/2023,drg,17072,MS-DRG 004,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,4,IP,NaN,NaN,na,Gross Charges,949211.27,4
17072,1/13/2023,drg,17073,MS-DRG 005,LIVER TRANSPLANT WITH MCC OR INTESTINAL TRANSP...,5,IP,NaN,NaN,na,Gross Charges,473562.45,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312803,1/13/2023,drg,19302,APR-DRG 952-2,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-2,IP,NaN,NaN,na,Workers Comp 50,NaN,952-2
1312804,1/13/2023,drg,19303,APR-DRG 952-3,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-3,IP,NaN,NaN,na,Workers Comp 50,NaN,952-3
1312805,1/13/2023,drg,19304,APR-DRG 952-4,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-4,IP,NaN,NaN,na,Workers Comp 50,NaN,952-4
1312806,1/13/2023,drg,19305,APR-DRG 955-1,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,955-1,IP,NaN,NaN,na,Workers Comp 50,NaN,955-1


In [14]:
df_mid_cdm = pd.DataFrame(df_mid.loc[df_mid['code_type'] == 'none'])
df_mid_cdm['code_prefix'] = 'none'
df_mid_cdm['code'] = 'na'
df_mid_cdm

,file_last_updated,code_type,internal_code,code_orig,description,drg,patient_class,cpt,modifier,rev_code,payer_orig,rate,code_prefix,code
0,1/13/2023,none,1,1000114S,CO-PILOT VALVE,NaN,OP,NaN,,0272,Gross Charges,150.0,none,na
1,1/13/2023,none,2,1000284S,SET LAP/CHOLANGIO,NaN,OP,NaN,,0272,Gross Charges,681.3,none,na
2,1/13/2023,none,3,1000564S,DRAINAGE BAG W/TUBING 600mL,NaN,OP,NaN,,0272,Gross Charges,97.5,none,na
3,1/13/2023,none,4,1000823S,INSTAT MCH MICROFIBLR CLGN 1GM,NaN,OP,NaN,,0272,Gross Charges,776.5,none,na
4,1/13/2023,none,5,1001512S,STAYFIX SMALL,NaN,OP,NaN,,0272,Gross Charges,65.63,none,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310564,1/13/2023,none,17063,9940S,SUTURE SILK 2-0 BLK BR 30 INCH,NaN,OP,NaN,,0272,Workers Comp 50,3.37,none,na
1310565,1/13/2023,none,17064,99478S,TAPER STRAIGHT 5FR 25CM,NaN,OP,NaN,,0272,Workers Comp 50,179.25,none,na
1310566,1/13/2023,none,17065,99479S,GLIDECTH 038in5F ANG TAPER65cm,NaN,OP,NaN,,0272,Workers Comp 50,192.75,none,na
1310568,1/13/2023,none,17067,999590S,SNARE POLYPECTOMY OPTIMIZER SM,NaN,OP,NaN,,0272,Workers Comp 50,37.31,none,na


In [15]:
def code_orig_to_code_prefix(code_orig):
    if code_orig.startswith('APR-DRG'):
        return 'apr-drg'
    elif code_orig.startswith('MS-DRG'):
        return 'ms-drg'
    
    return 'none'

def pad_drg_if_needed(drg):
    if len(drg) == 1:
        return "00" + drg
    elif len(drg) == 2:
        return "0" + drg

    return drg

df_mid_drg['code_type'] = 'drg'
df_mid_drg['code_prefix'] = df_mid['code_orig'].apply(code_orig_to_code_prefix)
df_mid_drg['code'] = df_mid_drg['drg'].apply(pad_drg_if_needed)
df_mid_drg

,file_last_updated,code_type,internal_code,code_orig,description,drg,patient_class,cpt,modifier,rev_code,payer_orig,rate,code,code_prefix
17068,1/13/2023,drg,17069,MS-DRG 001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,1,IP,NaN,NaN,na,Gross Charges,846592.03,001,ms-drg
17069,1/13/2023,drg,17070,MS-DRG 002,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,2,IP,NaN,NaN,na,Gross Charges,752886.92,002,ms-drg
17070,1/13/2023,drg,17071,MS-DRG 003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,3,IP,NaN,NaN,na,Gross Charges,645099.41,003,ms-drg
17071,1/13/2023,drg,17072,MS-DRG 004,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,4,IP,NaN,NaN,na,Gross Charges,949211.27,004,ms-drg
17072,1/13/2023,drg,17073,MS-DRG 005,LIVER TRANSPLANT WITH MCC OR INTESTINAL TRANSP...,5,IP,NaN,NaN,na,Gross Charges,473562.45,005,ms-drg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312803,1/13/2023,drg,19302,APR-DRG 952-2,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-2,IP,NaN,NaN,na,Workers Comp 50,NaN,952-2,apr-drg
1312804,1/13/2023,drg,19303,APR-DRG 952-3,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-3,IP,NaN,NaN,na,Workers Comp 50,NaN,952-3,apr-drg
1312805,1/13/2023,drg,19304,APR-DRG 952-4,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,952-4,IP,NaN,NaN,na,Workers Comp 50,NaN,952-4,apr-drg
1312806,1/13/2023,drg,19305,APR-DRG 955-1,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,955-1,IP,NaN,NaN,na,Workers Comp 50,NaN,955-1,apr-drg


In [16]:
df_mid = pd.concat([df_mid_cpt, df_mid_drg, df_mid_cdm])
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,drg,patient_class,cpt,modifier,rev_code,payer_orig,rate,code,code_prefix
8,1/13/2023,cpt,9,1001919S,DRESSING CENTRAL LINE 6 X 3 1/,NaN,OP,A6254,,0272,Gross Charges,3.32,A6254,hcpcs_cpt
21,1/13/2023,cpt,22,1002080S,TRAY URETHRAL CATHETER 15 FR,NaN,OP,A4353,,0272,Gross Charges,12.66,A4353,hcpcs_cpt
28,1/13/2023,cpt,29,1013481S,FELT TEFLON 6 X 6,NaN,IP,C1768,,0278,Gross Charges,315.0,C1768,hcpcs_cpt
31,1/13/2023,cpt,32,1013481S,FELT TEFLON 6 X 6,NaN,OP,C1768,,0278,Gross Charges,315.0,C1768,hcpcs_cpt
39,1/13/2023,cpt,40,1013841S,SHEATH 7FR,NaN,OP,C1894,,0272,Gross Charges,65.63,C1894,hcpcs_cpt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310564,1/13/2023,none,17063,9940S,SUTURE SILK 2-0 BLK BR 30 INCH,NaN,OP,NaN,,0272,Workers Comp 50,3.37,na,none
1310565,1/13/2023,none,17064,99478S,TAPER STRAIGHT 5FR 25CM,NaN,OP,NaN,,0272,Workers Comp 50,179.25,na,none
1310566,1/13/2023,none,17065,99479S,GLIDECTH 038in5F ANG TAPER65cm,NaN,OP,NaN,,0272,Workers Comp 50,192.75,na,none
1310568,1/13/2023,none,17067,999590S,SNARE POLYPECTOMY OPTIMIZER SM,NaN,OP,NaN,,0272,Workers Comp 50,37.31,na,none


In [17]:
del df_mid['drg']
del df_mid['cpt']
df_mid['modifier'] = df_mid['modifier'].fillna('na')
df_mid['modifier'] = df_mid['modifier'].apply(lambda modifier: 'na' if len(modifier.strip()) == 0 else modifier) 
df_mid['rev_code'] = df_mid['rev_code'].fillna('na')

In [18]:
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,patient_class,modifier,rev_code,payer_orig,rate,code,code_prefix
8,1/13/2023,cpt,9,1001919S,DRESSING CENTRAL LINE 6 X 3 1/,OP,na,0272,Gross Charges,3.32,A6254,hcpcs_cpt
21,1/13/2023,cpt,22,1002080S,TRAY URETHRAL CATHETER 15 FR,OP,na,0272,Gross Charges,12.66,A4353,hcpcs_cpt
28,1/13/2023,cpt,29,1013481S,FELT TEFLON 6 X 6,IP,na,0278,Gross Charges,315.0,C1768,hcpcs_cpt
31,1/13/2023,cpt,32,1013481S,FELT TEFLON 6 X 6,OP,na,0278,Gross Charges,315.0,C1768,hcpcs_cpt
39,1/13/2023,cpt,40,1013841S,SHEATH 7FR,OP,na,0272,Gross Charges,65.63,C1894,hcpcs_cpt
...,...,...,...,...,...,...,...,...,...,...,...,...
1310564,1/13/2023,none,17063,9940S,SUTURE SILK 2-0 BLK BR 30 INCH,OP,na,0272,Workers Comp 50,3.37,na,none
1310565,1/13/2023,none,17064,99478S,TAPER STRAIGHT 5FR 25CM,OP,na,0272,Workers Comp 50,179.25,na,none
1310566,1/13/2023,none,17065,99479S,GLIDECTH 038in5F ANG TAPER65cm,OP,na,0272,Workers Comp 50,192.75,na,none
1310568,1/13/2023,none,17067,999590S,SNARE POLYPECTOMY OPTIMIZER SM,OP,na,0272,Workers Comp 50,37.31,na,none


In [19]:
def payer_category_from_payer_orig(payer_orig):
    if payer_orig == "Gross Charges":
        return 'gross'
    elif payer_orig == "Discounted Cash Price":
        return 'cash'
    elif payer_orig == "De-Identified Minimum":
        return 'min'
    elif payer_orig == "De-Identified Maximum":
        return 'max'
    
    return 'payer'

df_mid['payer_category'] = df_mid['payer_orig'].apply(payer_category_from_payer_orig)
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,patient_class,modifier,rev_code,payer_orig,rate,code,code_prefix,payer_category
8,1/13/2023,cpt,9,1001919S,DRESSING CENTRAL LINE 6 X 3 1/,OP,na,0272,Gross Charges,3.32,A6254,hcpcs_cpt,gross
21,1/13/2023,cpt,22,1002080S,TRAY URETHRAL CATHETER 15 FR,OP,na,0272,Gross Charges,12.66,A4353,hcpcs_cpt,gross
28,1/13/2023,cpt,29,1013481S,FELT TEFLON 6 X 6,IP,na,0278,Gross Charges,315.0,C1768,hcpcs_cpt,gross
31,1/13/2023,cpt,32,1013481S,FELT TEFLON 6 X 6,OP,na,0278,Gross Charges,315.0,C1768,hcpcs_cpt,gross
39,1/13/2023,cpt,40,1013841S,SHEATH 7FR,OP,na,0272,Gross Charges,65.63,C1894,hcpcs_cpt,gross
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310564,1/13/2023,none,17063,9940S,SUTURE SILK 2-0 BLK BR 30 INCH,OP,na,0272,Workers Comp 50,3.37,na,none,payer
1310565,1/13/2023,none,17064,99478S,TAPER STRAIGHT 5FR 25CM,OP,na,0272,Workers Comp 50,179.25,na,none,payer
1310566,1/13/2023,none,17065,99479S,GLIDECTH 038in5F ANG TAPER65cm,OP,na,0272,Workers Comp 50,192.75,na,none,payer
1310568,1/13/2023,none,17067,999590S,SNARE POLYPECTOMY OPTIMIZER SM,OP,na,0272,Workers Comp 50,37.31,na,none,payer


In [20]:
df_mid['patient_class'] = df_mid['patient_class'].replace("OP", "outpatient").replace("IP", "inpatient")
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,patient_class,modifier,rev_code,payer_orig,rate,code,code_prefix,payer_category
8,1/13/2023,cpt,9,1001919S,DRESSING CENTRAL LINE 6 X 3 1/,outpatient,na,0272,Gross Charges,3.32,A6254,hcpcs_cpt,gross
21,1/13/2023,cpt,22,1002080S,TRAY URETHRAL CATHETER 15 FR,outpatient,na,0272,Gross Charges,12.66,A4353,hcpcs_cpt,gross
28,1/13/2023,cpt,29,1013481S,FELT TEFLON 6 X 6,inpatient,na,0278,Gross Charges,315.0,C1768,hcpcs_cpt,gross
31,1/13/2023,cpt,32,1013481S,FELT TEFLON 6 X 6,outpatient,na,0278,Gross Charges,315.0,C1768,hcpcs_cpt,gross
39,1/13/2023,cpt,40,1013841S,SHEATH 7FR,outpatient,na,0272,Gross Charges,65.63,C1894,hcpcs_cpt,gross
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310564,1/13/2023,none,17063,9940S,SUTURE SILK 2-0 BLK BR 30 INCH,outpatient,na,0272,Workers Comp 50,3.37,na,none,payer
1310565,1/13/2023,none,17064,99478S,TAPER STRAIGHT 5FR 25CM,outpatient,na,0272,Workers Comp 50,179.25,na,none,payer
1310566,1/13/2023,none,17065,99479S,GLIDECTH 038in5F ANG TAPER65cm,outpatient,na,0272,Workers Comp 50,192.75,na,none,payer
1310568,1/13/2023,none,17067,999590S,SNARE POLYPECTOMY OPTIMIZER SM,outpatient,na,0272,Workers Comp 50,37.31,na,none,payer


In [21]:
df_mid['billing_class'] = 'na'

In [22]:
def plan_type_from_payer_orig(payer_orig):
    if "PPO" in payer_orig:
        return "PPO"
    elif "HMO" in payer_orig:
        return "HMO"
    
    return None

df_mid['plan_type'] = df_mid['payer_orig'].apply(plan_type_from_payer_orig)
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,patient_class,modifier,rev_code,payer_orig,rate,code,code_prefix,payer_category,billing_class,plan_type
8,1/13/2023,cpt,9,1001919S,DRESSING CENTRAL LINE 6 X 3 1/,outpatient,na,0272,Gross Charges,3.32,A6254,hcpcs_cpt,gross,na,None
21,1/13/2023,cpt,22,1002080S,TRAY URETHRAL CATHETER 15 FR,outpatient,na,0272,Gross Charges,12.66,A4353,hcpcs_cpt,gross,na,None
28,1/13/2023,cpt,29,1013481S,FELT TEFLON 6 X 6,inpatient,na,0278,Gross Charges,315.0,C1768,hcpcs_cpt,gross,na,None
31,1/13/2023,cpt,32,1013481S,FELT TEFLON 6 X 6,outpatient,na,0278,Gross Charges,315.0,C1768,hcpcs_cpt,gross,na,None
39,1/13/2023,cpt,40,1013841S,SHEATH 7FR,outpatient,na,0272,Gross Charges,65.63,C1894,hcpcs_cpt,gross,na,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310564,1/13/2023,none,17063,9940S,SUTURE SILK 2-0 BLK BR 30 INCH,outpatient,na,0272,Workers Comp 50,3.37,na,none,payer,na,None
1310565,1/13/2023,none,17064,99478S,TAPER STRAIGHT 5FR 25CM,outpatient,na,0272,Workers Comp 50,179.25,na,none,payer,na,None
1310566,1/13/2023,none,17065,99479S,GLIDECTH 038in5F ANG TAPER65cm,outpatient,na,0272,Workers Comp 50,192.75,na,none,payer,na,None
1310568,1/13/2023,none,17067,999590S,SNARE POLYPECTOMY OPTIMIZER SM,outpatient,na,0272,Workers Comp 50,37.31,na,none,payer,na,None


In [23]:
df_mid['code_orig'] = None
df_mid['plan_orig'] = 'na'
df_mid['hospital_ccn'] = ccn
df_mid['hospital_ein'] = ein
df_mid['filename'] = filename

from dateutil.parser import parse as parse_datetime

df_mid['file_last_updated'] = df_mid['file_last_updated'].apply(
    lambda file_last_updated: parse_datetime(file_last_updated).isoformat().split("T")[0])

df_mid['url'] = url
df_mid['internal_code'] = df_mid['internal_code'].fillna('na')
df_mid['ndc'] = 'na'
df_mid['rate'] = df_mid['rate'].apply(lambda rate: None if type(rate) == str and rate.startswith("ERROR") else rate)
df_mid

,file_last_updated,code_type,internal_code,code_orig,description,patient_class,modifier,rev_code,payer_orig,rate,...,code_prefix,payer_category,billing_class,plan_type,plan_orig,hospital_ccn,hospital_ein,filename,url,ndc
8,2023-01-13,cpt,9,None,DRESSING CENTRAL LINE 6 X 3 1/,outpatient,na,0272,Gross Charges,3.32,...,hcpcs_cpt,gross,na,None,na,030037,62-1795587,621795587_TempeStLukesHospital_standardcharges...,https://content.steward.org/sites/default/file...,na
21,2023-01-13,cpt,22,None,TRAY URETHRAL CATHETER 15 FR,outpatient,na,0272,Gross Charges,12.66,...,hcpcs_cpt,gross,na,None,na,030037,62-1795587,621795587_TempeStLukesHospital_standardcharges...,https://content.steward.org/sites/default/file...,na
28,2023-01-13,cpt,29,None,FELT TEFLON 6 X 6,inpatient,na,0278,Gross Charges,315.0,...,hcpcs_cpt,gross,na,None,na,030037,62-1795587,621795587_TempeStLukesHospital_standardcharges...,https://content.steward.org/sites/default/file...,na
31,2023-01-13,cpt,32,None,FELT TEFLON 6 X 6,outpatient,na,0278,Gross Charges,315.0,...,hcpcs_cpt,gross,na,None,na,030037,62-1795587,621795587_TempeStLukesHospital_standardcharges...,https://content.steward.org/sites/default/file...,na
39,2023-01-13,cpt,40,None,SHEATH 7FR,outpatient,na,0272,Gross Charges,65.63,...,hcpcs_cpt,gross,na,None,na,030037,62-1795587,621795587_TempeStLukesHospital_standardcharges...,https://content.steward.org/sites/default/file...,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310564,2023-01-13,none,17063,None,SUTURE SILK 2-0 BLK BR 30 INCH,outpatient,na,0272,Workers Comp 50,3.37,...,none,payer,na,None,na,030037,62-1795587,621795587_TempeStLukesHospital_standardcharges...,https://content.steward.org/sites/default/file...,na
1310565,2023-01-13,none,17064,None,TAPER STRAIGHT 5FR 25CM,outpatient,na,0272,Workers Comp 50,179.25,...,none,payer,na,None,na,030037,62-1795587,621795587_TempeStLukesHospital_standardcharges...,https://content.steward.org/sites/default/file...,na
1310566,2023-01-13,none,17065,None,GLIDECTH 038in5F ANG TAPER65cm,outpatient,na,0272,Workers Comp 50,192.75,...,none,payer,na,None,na,030037,62-1795587,621795587_TempeStLukesHospital_standardcharges...,https://content.steward.org/sites/default/file...,na
1310568,2023-01-13,none,17067,None,SNARE POLYPECTOMY OPTIMIZER SM,outpatient,na,0272,Workers Comp 50,37.31,...,none,payer,na,None,na,030037,62-1795587,621795587_TempeStLukesHospital_standardcharges...,https://content.steward.org/sites/default/file...,na


In [24]:
df_out = pd.DataFrame(columns=TARGET_COLUMNS)
df_out = df_out.append(df_mid)
df_out = df_out.dropna(subset=['rate'], axis=0)
df_out

/tmp/ipykernel_36578/1393279649.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_out = df_out.append(df_mid)


,hospital_ccn,hospital_ein,code_type,internal_code,description,code_orig,code_prefix,code,modifier,ndc,...,plan_name,plan_id,plan_type,rate,rate_method,rate_desc,filename,file_last_updated,url,permalink
8,030037,62-1795587,cpt,9,DRESSING CENTRAL LINE 6 X 3 1/,None,hcpcs_cpt,A6254,na,na,...,NaN,NaN,None,3.32,NaN,NaN,621795587_TempeStLukesHospital_standardcharges...,2023-01-13,https://content.steward.org/sites/default/file...,NaN
21,030037,62-1795587,cpt,22,TRAY URETHRAL CATHETER 15 FR,None,hcpcs_cpt,A4353,na,na,...,NaN,NaN,None,12.66,NaN,NaN,621795587_TempeStLukesHospital_standardcharges...,2023-01-13,https://content.steward.org/sites/default/file...,NaN
28,030037,62-1795587,cpt,29,FELT TEFLON 6 X 6,None,hcpcs_cpt,C1768,na,na,...,NaN,NaN,None,315.0,NaN,NaN,621795587_TempeStLukesHospital_standardcharges...,2023-01-13,https://content.steward.org/sites/default/file...,NaN
31,030037,62-1795587,cpt,32,FELT TEFLON 6 X 6,None,hcpcs_cpt,C1768,na,na,...,NaN,NaN,None,315.0,NaN,NaN,621795587_TempeStLukesHospital_standardcharges...,2023-01-13,https://content.steward.org/sites/default/file...,NaN
39,030037,62-1795587,cpt,40,SHEATH 7FR,None,hcpcs_cpt,C1894,na,na,...,NaN,NaN,None,65.63,NaN,NaN,621795587_TempeStLukesHospital_standardcharges...,2023-01-13,https://content.steward.org/sites/default/file...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310564,030037,62-1795587,none,17063,SUTURE SILK 2-0 BLK BR 30 INCH,None,none,na,na,na,...,NaN,NaN,None,3.37,NaN,NaN,621795587_TempeStLukesHospital_standardcharges...,2023-01-13,https://content.steward.org/sites/default/file...,NaN
1310565,030037,62-1795587,none,17064,TAPER STRAIGHT 5FR 25CM,None,none,na,na,na,...,NaN,NaN,None,179.25,NaN,NaN,621795587_TempeStLukesHospital_standardcharges...,2023-01-13,https://content.steward.org/sites/default/file...,NaN
1310566,030037,62-1795587,none,17065,GLIDECTH 038in5F ANG TAPER65cm,None,none,na,na,na,...,NaN,NaN,None,192.75,NaN,NaN,621795587_TempeStLukesHospital_standardcharges...,2023-01-13,https://content.steward.org/sites/default/file...,NaN
1310568,030037,62-1795587,none,17067,SNARE POLYPECTOMY OPTIMIZER SM,None,none,na,na,na,...,NaN,NaN,None,37.31,NaN,NaN,621795587_TempeStLukesHospital_standardcharges...,2023-01-13,https://content.steward.org/sites/default/file...,NaN


In [25]:
df_out.to_csv(ccn + ".csv", index=False)

In [26]:
df_in[df_in['Charge Code/Package'] == '7908377S']

,As of Date,Line Type,Line ID,Charge Code/Package,Charge Description,DRG,Patient Type,CPT,Modifiers,Rev Code,...,Tricare HNFS,TriWest VA PCCC,United Behavioral Health,United Healthcare Compass HIX,United Healthcare Evercare AHCCCS,United Healthcare Medicare Advantage,United Healthcare No Options (All Other),University Physicians Healthcare Medicare Advantage,Workers Comp 100,Workers Comp 50
6098,1/13/2023,CDM,6099,7908377S,CATH IMAGER II BERN .038 5FX10,NaN,OP,NaN,,272,...,81.29,NaN,81.29,38.78,NaN,NaN,38.78,NaN,81.29,40.65
6099,1/13/2023,CDM,6100,7908377S,CATH IMAGER II BERN .038 5FX10,NaN,OP,53.59,,272,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
